In [ ]:
from IPython.display import YouTubeVideo, Markdown, SVG
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

display(Markdown(filename='../../preamble.md'))

In [ ]:
%autosave 0
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import pandas as pd

from IPython.display import display
import ipywidgets as widgets
from functools import partial
slider_layout = widgets.Layout(width='600px', height='20px')
slider_style = {'description_width': 'initial'}
IntSlider_nice = partial(widgets.IntSlider, style=slider_style, layout=slider_layout, continuous_update=False)
FloatSlider_nice = partial(widgets.FloatSlider, style=slider_style, layout=slider_layout, continuous_update=False)
SelSlider_nice = partial(widgets.SelectionSlider, style=slider_style, layout=slider_layout, continuous_update=False)
from scipy.special import erf
gaussian_pdf = lambda x, mu=0, s=1: np.exp(-0.5*(x-mu)**2/s**2)/(s*np.sqrt(2*np.pi))
gaussian_cdf = lambda x, mu=0, s=1: 0.5 + 0.5*erf((x-mu)/(s*np.sqrt(2)))

Luego de aplicar las herramientas de estadísticas descriptiva para entender mejor nuestros datos podemos enfocarnos en

# Inferencia estadística

La inferencia busca

> Extraer conclusiones a partir de hechos a través de un método o premisa

En el caso particular de la inferencia estadística tenemos que

- Hechos: Datos
- Premisa: Modelo probabilístico
- Conclusión: Una cantidad no observada que es interesante

Y lo que buscamos es

> Cuantificar la incerteza de la conclusión dado los datos y el modelo 

La inferencia estadística puede dividirse en los siguientes tres niveles
1. Ajustar un modelo a nuestros datos: **Máxima verosimilitud**
1. Verificar que el modelo sea confiable: **Intervalo de confianza**
1. Responder una pregunta usando el modelo: **Test de hipótesis**

A continuación estudiaremos cada una de estas herramientas

## Ajuste de modelos paramétricos

Se refiere a aquellos modelos que explicitan una distribución de probabilidad

### Ejemplo: La pesa defectuosa

- Mi pesa está defectuosa
- Luego de comer mido $M$ veces mi peso obteniendo un conjunto de observaciones $\{x_i\}$
- El objetivo es encontrar mi peso real $\hat x$. 

Puedo modelar mis observaciones como
$$
x_i = \hat x + \varepsilon_i
$$
donde $\varepsilon_i$ corresponde al ruido del instrumento

Asumamos que $\varepsilon_i \sim \mathcal{N}(0, \sigma_\varepsilon^2)$

es decir que el ruido es
- Independiente
- Gaussiano
- Con media cero y varianza $\sigma_\varepsilon^2$

Con esto podemos escribir la probabilidad de observar $x_i$ dado un cierto valor $\hat x$ como

$$
p(x_i|\hat x) = \mathcal{N}(\hat x, \sigma_\varepsilon^2)
$$

> Mi modelo tiene dos parámetros: $\hat x$ y $\sigma_\varepsilon^2$

Si **asumimos** que las observaciones son independientes e identicamente distribuidas (iid), podemos escribir la probabilidad de observar el conjunto completo como 

$$
\begin{align}
p(x_1, x_2, \ldots, x_M| \hat x) &= \prod_{i=1}^M p(x_i|\hat x) \nonumber \\
&= \prod_{i=1}^M  \frac{1}{\sqrt{2\pi}\sigma_\varepsilon}  \exp \left ( - \frac{1}{2\sigma_\varepsilon^2} (x_i - \hat x)^2 \right) = \mathcal{L}(\hat x)  \nonumber
\end{align}
$$

> Esto se conoce como la **verosimilitud** de los parámetros dado el conjunto de observaciones

Podemos buscar los parámetros que hacen mi modelo más "creible" **maximizando la verosimilitud**


$$
\max_{\hat x}  \mathcal{L}(\hat x) = \max_{\hat x}  \log \mathcal{L}(\hat x) = -\frac{1}{2} \log(2\pi\sigma_\varepsilon^2)M - \frac{1}{2\sigma_\varepsilon^2} \sum_{i=1}^M (x_i - \hat x )^2
$$

Como el logaritmo es monotónico conviene maximizar el log de la verosimilitud

Si derivamos e igualamos a cero

$$
\begin{align}
-\frac{1}{2\sigma_\varepsilon^2} \sum_{i=1}^M 2(x_i - \hat x ) &= 0 \nonumber \\
\hat x &= \frac{1}{M} \sum_{i=1}^M x_i \nonumber 
\end{align}
$$

> El estimador de máxima verosimilitud para la media de una Gaussiana es el clásico promedio muestral

Si hicieramos lo mismo para la varianza encontraríamos

$$
\sigma_\epsilon^2 = \frac{1}{M} \sum_{i=1}^M (x_i - \hat x)^2
$$

> Que es la varianza muestreal (sesgada)

# Estimación de máxima verosimilitud

El procedimiento que acabamos de ver se llama *maximum likelihood estimation* (MLE)

Procedimiento

1. Definir los supuestos del problema, el modelo (distribución) y sus parámetros $\theta$
1. Escribir el **logaritmo de la verosimilud** de los parámetros $\log \mathcal{L}(\theta)$
1. Encontrar $\theta$ que maximiza 
$$
\hat \theta = \text{arg}\max_\theta \log \mathcal{L}(\theta)
$$


Las distribuciones de  `scipy.stats` tienen los métodos
- `fit (data)` Ajusta una distribución continua a datos usando MLE
- `expect (func)` Valor esperado de una función c/r a la distribución
- `interval (alpha)` Cotas para el intervalo que contiene un porcentaje $\alpha$ de la distribución

Para distribuciones complicadas sin solución analítica se pueden usar métodos iterativos

### Ejemplo: Ajustando y comparando distintas distribuciones

Observemos la siguiente distribución, ¿Qué características resaltan? ¿Qué distribución sería apropiado ajustar?

In [ ]:
from sklearn import datasets
data_set = datasets.load_breast_cancer()
x, y = data_set['data'][:, 0], data_set['target']
fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
ax.hist(x, bins=20, density=True)
ax.set_xlabel('Tamaño del nodulo');

In [ ]:
from collections import Counter
Counter(y)

Podemos probar varias distribuciones y observar el resultado

¿Cómo medir la bondad del ajuste?

In [ ]:
x_plot = np.linspace(np.amin(x), np.amax(x), num=500)
dist = scipy.stats.uniform
params = dist.fit(x)
print(dist.name)
print(params)
p_plot = dist(*params[:-2], loc=params[-2], scale=params[-1]).pdf(x_plot)
ax.plot(x_plot, p_plot, label=dist.name)
plt.legend();

### Bondad de ajuste

Podemos usar el test chi-cuadrado, el [test de Akaike](https://en.wikipedia.org/wiki/Akaike_information_criterion), el test no-paramétrico de Kolmogorov-Smirnov (KS) o gráficos QQ para medir que tan bien se ajusta nuestra distribución teórica a los datos

El test KS nos permite comparar que tan distinta es una distribución continua empírica de una teórica comparando sus CDFs: [`scipy.stats.kstest`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html)

- Require que los datos estén estandarizados 

$$
Z = \frac{X - \mu_X}{\sigma_X}
$$

- Retorna un estadístico: Mientras más cerca a cero, mejor es el ajuste
- También retorna un p-value: Si es menor que $\alpha$ entonces rechazo la hipótesis nula de que las distribuciones son iguales con un $1-\alpha$ de confianza. Veremos test de hipótesis más adelante


In [ ]:
x_std = (x-np.mean(x))/np.std(x)
ks_res = []
for dist in [scipy.stats.norm, scipy.stats.lognorm, scipy.stats.beta, scipy.stats.gamma, scipy.stats.genextreme]:    
    params = dist.fit(x_std)
    fitted_dist = dist(*params[:-2], loc=params[-2], scale=params[-1])
    ks_res.append(scipy.stats.kstest(rvs=x_std, cdf=fitted_dist.cdf))
    print(dist.name)
    print(ks_res[-1])

### Respondiendo preguntas usando nuestro modelo

1. ¿Cuál es la media de la distribución? 
1. ¿Cúal es la probabilidad de que el "tamaño del nódulo" sea mayor o igual que 20?
1. ¿Cúal es el tamaño de nódulo que acumula el 90% de la distribución?

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
ax.hist(x, bins=20, density=True)
ax.set_xlabel('Tamaño del nodulo')
dist = scipy.stats.genextreme
params = dist.fit(x)
fitted_dist = dist(*params[:-2], loc=params[-2], scale=params[-1])
p_plot = fitted_dist.pdf(x_plot)
ax.plot(x_plot, p_plot, c='k', lw=2);
ax.fill_between(x_plot[x_plot>=20], np.zeros_like(x_plot[x_plot>=20]), p_plot[x_plot>=20], 
                alpha=0.5, color='k', zorder=20)

# P1: Usando el atributo mean() 
display(fitted_dist.mean())
# P2: Usando el atributo CDF
display(1. - fitted_dist.cdf(20))
# P3: Usando el atributo PPF
display(fitted_dist.ppf(0.9))

p-values: https://ipython-books.github.io/72-getting-started-with-statistical-hypothesis-testing-a-simple-z-test/

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3), tight_layout=True)
ax.hist(x[y==0], bins=30, range=(5, 30), density=False, alpha=0.75, label='maligno')
ax.hist(x[y==1], bins=30, range=(5, 30), density=False, alpha=0.75, label='benigno')
ax.set_xlabel('Tamaño del nodulo');
plt.legend();

### Ejemplo: Ajustando una recta

Sea el siguiente dataset de [consumo de helados](https://forge.scilab.org/index.php/p/rdataset/source/tree/master/csv/Ecdat/Icecream.csv) en USA

In [ ]:
!wget -nc -c https://forge.scilab.org/index.php/p/rdataset/source/file/master/csv/Ecdat/Icecream.csv
df = pd.read_csv('Icecream.csv', header=0, index_col=0)
df.columns = ['consumo', 'ingreso', 'precio', 'temperatura']
display(df.head())

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True, sharey=True)
for i, col in enumerate(df.columns[1:]):
    ax[i].scatter(df[col], df["consumo"])
    ax[i].set_xlabel(col)
ax[0].set_ylabel(df.columns[0]);

Sea $y$ el consumo y $x$ la temperatura.

Asumiendo errores gaussianos

$$
y_i = \hat y_i + \epsilon_i, \epsilon_i \sim \mathcal{N}(0, \sigma^2),
$$

un modelo lineal de dos parámetros,

$$
\hat y_i = \theta_0 + \theta_1 x_i
$$

y observaciones iid, podemos estimar $\theta$ buscando la máxima verosimilitud del modelo

$$
\min_\theta \log \mathcal{L}(\theta) = \sum_{i=1}^M (y_i - \theta_0 - \theta_1 x_i)^2
$$

donde 
$$
\sum_i y_i  - M\theta_0 - \theta_1  \sum_i x_i = 0 \rightarrow \theta_0 = \bar y - \theta_1 \bar x
$$
$$
\sum_i y_i x_i - \theta_0 \sum_i x_i - \theta_1 \sum_i x_i^2 = 0
$$

$$
\theta_1 = \frac{\sum_i x_i y_i - M \bar x \bar y}{\sum_i x_i^2 - M \bar x^2} = \frac{ \sum_i (y_i - \bar y)(x_i - \bar x)}{\sum_i (x_i - \bar x)^2} = \frac{\text{COV}(x, y)}{\text{Var}(x)}
$$

la fuerza de la correlación se mide con 

$$
r^2 = 1 - \frac{\sum_i ( y_i - \hat y_i)^2}{\sum_i ( y_i - \bar y)^2} = 1 - \frac{\frac{1}{M} \sum_i (y_i - \hat y_i)^2}{\text{Var}(y)} = \frac{\text{COV}^2(x, y)}{\text{Var}(x) \text{Var}(y)}
$$

donde $r = \frac{\text{COV}(x, y)}{\sqrt{\text{Var}(x) \text{Var}(y)}} \in [-1, 1]$ se conoce como [coeficiente de correlación de Pearson](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)

Podemos usar pandas para calcular $r$

In [ ]:
df.corr()

Es decir que existe 
- una correlación positiva alta entre consumo y temperatura
- una correlación negativa moderada entre consumo y precio
- una correlación cercana a cero entre consumo e ingreso

Podemos usar la función [`scipy.stats.linregress`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html) para recuperar $\theta_0$, $\theta_1$ y $r$

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True, sharey=True)
ax[0].set_ylabel(df.columns[0]);

for i, col in enumerate(df.columns[1:]):
    ax[i].scatter(df[col], df["consumo"])    
    res = scipy.stats.linregress(df[col], df["consumo"])
    ax[i].set_title("r: {0:0.5f}".format(res.rvalue))
    x_plot = np.linspace(np.amin(df[col]), np. amax(df[col]), num=100)
    ax[i].plot(x_plot, res.slope*x_plot + res.intercept, lw=4, alpha=0.5, c='k');
    ax[i].set_xlabel(col)

### Respondiendo preguntas con nuestro modelo

1. ¿Qué tan confiables son los valores de $\theta_0$, $\theta_1$ y $r$?
1. ¿Son las correlaciones encontradas significativas?


Dos caminos para responder
- Pruebas paramétricas
- Bootstrap

## Test de hipótesis

Se aplica un tratamiento nuevo a una muestra de la población 
- ¿Es el tratamiento efectivo?
- ¿Existe una diferencia entre los que tomaron el tratamiento y los que no?

El test de hipótesis es un procedimiento estadístico para comprobar si el resultado de un experimento es significativo en la población

Para esto formulamos dos escenarios cada uno con una hipótesis asociada
- Hipótesis nula ($H_0$): El experimento no produjo diferencia. El experimento no tuvo efecto. Las observaciones son producto del azar
- Hipótesis alternativa ($H_A$): Usualmente el complemento de $H_0$

El test de hipótesis se diseña para medir que tan fuerte es la evidencia **en contra** de la hipótesis nula

El algoritmo general es de un test de hipótesis:
1. Definimos $H_0$ y $H_A$
1. Definimos un estadístico $T$
1. Asumimos una distribución para $T$ dado que $H_0$ es cierto
1. Seleccionamos un nivel de significancia $\alpha$ 
1. Calculamos el $T$ para nuestros datos $T_{data}$
1. Calculamos el **p-value**: Probabilidad de observar un valor de $T$ más extremo que el observado 
    - Si nuestro test es de una cola:
        - Superior: $p = P(T>T_{data})$
        - Inferior: $p = P(T<T_{data})$
    - Si nuestro test es dos colas: $p = P(T>T_{data}) + P(T<T_{data})$

Finalmente:

`Si`  $p < \alpha$
    
    Rechazamos la hipótesis nula con confianza (1-\alpha)
De lo contrario:
    
    No hay suficiente evidencia para rechazar la hipótesis nula
    
**Escenarios:**
- Rechazamos $H_0$ cuando en realidad era cierta (falso positivo): **Error tipo I**
    - Ocurre con probabilidad $\alpha$ (es controlable)
- No rechazamos $H_0$ cuando en realidad era falsa (falso negativo): **Error tipo II**
    - Ocurre con probabilidad $\beta$


**Errores de interpretación comunes:**

- El p-value **no es** la probabilidad de que $H_0$ sea cierta

    $p = P(T> T_{data} | H_0) \neq P(H_0 | T> T_{data})$
    
- No rechazar $H_0$ no es lo mismo que aceptarla
- Rechazar $H_0$ no es lo mismo que aceptar $H_A$

### Ejemplo: t-test de una muestra 

Para un conjunto de $M$ observaciones iid $X = {x_1, x_2, \ldots, x_M}$ con media muestral $\bar x = \sum_{i=1}^M x_i$ 

El t-test de una muestra es un test de hipótesis que busca probar si $\bar x$ es significativamente distinta de la **media poblacional** $\mu$, en el caso de que **no conocemos la varianza poblacional** $\sigma^2$

Las hipótesis son

- $H_0:$ $\bar x = \mu$
- $H_A:$ $\bar x \neq \mu$ (dos colas)

El estadístico de prueba es 

$$
t = \frac{\bar x - \mu}{\hat \sigma /\sqrt{M-1}}
$$

donde $\hat \sigma = \sqrt{ \frac{1}{M} \sum_{i=1}^M (x_i - \bar x)^2}$ es la desviación estándar muestral (sesgada)

Si asumimos que $\bar x$ se distribuye $\mathcal{N}(\mu, \frac{\sigma^2}{M})$ entonces
$t$ que se distribuye [t-student](https://en.wikipedia.org/wiki/Student%27s_t-distribution) con $M-1$ grados de libertad

- Para muestras iid y $M$ grande el supuesto se cumple por teorema central del límite
- Si $M$ es pequeño debemos verificar la normalidad de los datos

### t-test para probar que la regresión es significativa

En este caso queremos hacer el siguiente test
- $H_0:$ La pendiente es nula $\theta_1= 0$ 
- $H_A:$ La pendiente no es nula: $\theta_1\neq 0$ (dos colas)

Asumimos que $\theta_1$ es normal pero que desconocemos su varianza 

Entonces se puede formular el siguiente estadístico de prueba 

$$
t = \frac{(\theta_1-\theta^*) }{\text{SE}_{\theta_1}/\sqrt{M-2}} = \frac{ r\sqrt{M-2}}{\sqrt{1-r^2}},
$$

donde en este caso particular se usa $\theta^*=0$ y $\text{SE}_{\theta_1} = \sqrt{ \frac{\frac{1}{M} \sum_i (y_i - \hat y_i)^2}{\text{Var}(x)}}$

$t$ se [t-student](https://en.wikipedia.org/wiki/Student%27s_t-distribution) con dos grados de libertad (modelo de dos parámetros) 

La función `scipy.stats.linregress` implementa este test y retorna los p-values

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(8, 3), tight_layout=True, sharey=True)
t = np.linspace(-7, 7, num=1000)
M = df.shape[0]
ax[0].set_ylabel(df.columns[0]);

for i, col in enumerate(df.columns[1:]):
    res = scipy.stats.linregress(df[col], df["consumo"])
    t_data = res.rvalue*np.sqrt(M-2)/np.sqrt(1.-res.rvalue**2)
    display(res)
    ax[i].set_title("t_data: {0:0.5f}".format(t_data))
    ax[i].set_xlabel(col)
    dist = scipy.stats.t(loc=0, scale=1, df=M-2)
    p = dist.pdf(t)
    ax[i].plot(t, p)
    ax[i].plot([dist.ppf(0.025)]*2, [0, np.amax(p)], 'k--')
    ax[i].plot([dist.ppf(0.975)]*2, [0, np.amax(p)], 'k--')
    ax[i].plot([t_data]*2, [0, np.amax(p)])

¿Qué podemos decir de las correlaciones con el consumo de helados?

Para temperatura y considerando $\alpha = 0.05$:

> $\text{p-value} = 4.7892e-07 < \alpha$

Por ende rechazamos $H_0$ con 95% de confianza

En cambio para ingreso y precio no podemos rechazar $H_0$

### Reflexión
- ¿Cómo se escogen el estadístico y la distribución de prueba? Depende del problema 
- ¿Qué prueba puedo usar si quiero hacer regresión lineal multivariada? [ANOVA](https://pythonfordatascience.org/anova-python/)
- ¿Qué pasa si mis datos tienen una relación que no es lineal? La prueba no es confiable
- ¿Qué pasa si $\theta_1$ no es normal? La prueba no es confiable
- ¿Qué pasa si el ruido no es Gaussiano? La prueba no es confiable
- ¿Qué pasa si el ruido es Gaussiano pero su varianza cambia en el tiempo? La prueba no es confiable

## Prueba no-paramétrica: *Bootstrap*

Podemos estimar la incerteza de un estimador de forma no-paramétrica usando **muestreo tipo *bootstrap***

Tomamos nuestros conjunto de datos de tamaño $M$ y creamos $T$ nuevos conjuntos que "se le parezcan" 

Luego se calcula el valor del estimador en esos nuevos conjuntos

*Bootstrap* está basado en la Ley de los grandes números y Teorema central del límite 

<img src="https://www.statisticshowto.datasciencecentral.com/wp-content/uploads/2016/10/bootstrap-sample.png">

- En este caso supondremos independencia: **Muestreo con reemplazo de tamaño $M$**
- También existe bootstrap basado en los residuos y bootstrap dependiente

Podemos usar la función [`numpy.random.choice`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.choice.html) para generar los nuevos índices

In [ ]:
plt.close('all'); fig, ax = plt.subplots(2, 2, figsize=(6, 5), tight_layout=True)
x, y = df["temperatura"].values, df["consumo"].values
x_plot = np.linspace(np.amin(x), np. amax(x), num=100)
res_all = scipy.stats.linregress(x, y)
M = len(x)

def update_plot(T):
    [ax_.cla() for ax_ in ax.ravel()]
    ax[0, 0].scatter(x, y, zorder=100, s=10)
    np.random.seed(0)
    param = np.zeros(shape=(T, 3))
    lines = []
    for t in range(T):
        bootstrap_idx = np.random.choice(np.arange(len(x)), size=len(x), replace=True)
        res = scipy.stats.linregress(x[bootstrap_idx], y[bootstrap_idx])
        lines.append(x_plot*res.slope + res.intercept)
        #ax[0, 0].plot(x_plot, x_plot*res.slope + res.intercept, alpha=0.05, c='k')
        param[t, :] = [res.intercept, res.slope, res.rvalue]
    ax[0, 0].plot(x_plot, x_plot*res_all.slope + res_all.intercept, alpha=1, c='r')
    lines = np.stack(lines)
    ax[0, 0].fill_between(x_plot, np.mean(lines, axis=0)-3*np.std(lines, axis=0), 
                          np.mean(lines, axis=0)+3*np.std(lines, axis=0), color='r', alpha=0.25)
    hist_val, hist_lim, _ = ax[0, 1].hist(param[:, 2], bins=15, density=True)
    ax[0, 1].plot([res_all.rvalue]*2, [0, np.max(hist_val)], c='r')
    ax[0, 1].set_xlabel('r')
    display("Intervalo de confianza al 95% de r {}".format(np.percentile(param[:, 2], [2.5, 97.5])))
    hist_val, hist_lim, _ = ax[1, 0].hist(param[:, 0], bins=15, density=True)
    ax[1, 0].plot([res_all.intercept]*2, [0, np.max(hist_val)], c='r')
    ax[1, 0].set_xlabel(r'$\theta_0$')
    display("Intervalo de confianza al 95% de theta_0 {}".format(np.percentile(param[:, 0], [2.5, 97.5])))
    hist_val, hist_lim, _ = ax[1, 1].hist(param[:, 1], bins=15, density=True)
    ax[1, 1].plot([res_all.slope]*2, [0, np.max(hist_val)], c='r')
    ax[1, 1].set_xlabel(r'$\theta_1$')
    display("Intervalo de confianza al 95% de theta_1 {}".format(np.percentile(param[:, 1], [2.5, 97.5])))

widgets.interact(update_plot, T=SelSlider_nice(options=[10, 20, 50, 100, 200, 500], value=50));

- $\theta_1=0$ no está en el intervalo de confianza por lo que podemos rechazar $H_0$
- Notamos que $\theta_0$ y $\theta_1$ son asintóticamente normales. El supuesto del t-test estaba bien!
- Si hubieramos hecho un t-test sobre $r$ no sería valido (no normalidad)

Más sobre [*bootstrap* y regresión lineal](https://www.stat.cmu.edu/~cshalizi/402/lectures/08-bootstrap/lecture-08.pdf) [aquí](http://homepage.divms.uiowa.edu/~rdecook/stat3200/notes/bootstrap_4pp.pdf) y [acá](https://www.sagepub.com/sites/default/files/upm-binaries/21122_Chapter_21.pdf)

In [ ]:
!rm Icecream.csv

### Ejemplo: Regresión lineal multivariada y mínimos cuadrados

Queremos ajustar un modelo lineal de $N$ parámetros a un conjunto de $M$ observaciones ruidosas

Podemos modelar 

$$
y_i = \Phi(x_i) \theta + \varepsilon_i
$$

donde $\varepsilon_i \sim \mathcal{N}(0, \sigma_\varepsilon^2)$

Si asumimos que los datos son iid podemos escribir el logaritmo de la verosimilitud como

$$
\begin{align}
\log p(y_1, y_2, \ldots, y_M| \theta) &= \log \prod_{i=1}^M p(y_i|\theta) \nonumber \\
&=  -\frac{1}{2} \log(2\pi\sigma_\varepsilon^2) M - \frac{1}{2\sigma_\varepsilon^2} \sum_{i=1}^M (y_i - \Phi(x_i) \theta)^2
\end{align}
$$

el primer término no depende de $\theta$ y el segundo es negativo

El resultado de maximizar la log verosimilitud es equivalente a
$$
\min_\theta \log \mathcal{L}(\theta) = \sum_{i=1}^M (y_i - \Phi(x_i) \theta)^2
$$

Que corresponde al problema de **mínimos cuadrados**

#### Reflexionemos
¿Qué estamos asumiendo cuando usamos mínimos cuadrados?

# Para leer y reflexionar: 

https://www.sciencedirect.com/science/article/pii/S0167715218300737